In [1]:
import rsmf
formatter = rsmf.setup(r"\documentclass[10pt]{iopart}")

import numpy as np
from matplotlib import pyplot as plt


# Simulation results of the three approaches

The simulation results of solving Max-Cut on $100$ Erdos-Renyi graphs are listed in `sim-results.json`.
For each entry, we give the random seed corresponding to that graph, and the approximation ratio achieved by three methods, among which `cp`, `qiq` and `gw` stands for our method, QAOA-in-QAOA and Goemans-Williamson respectively.

The following cell caintains the code used to generate the plot comparing three methods.

In [2]:
import json

data = json.load(open('data/sim-results.json'))

plt.cla()

cq, qiq, gw = np.zeros(len(data)), np.zeros(len(data)), np.zeros(len(data))
for item in data:
    cq[item['seed']] = item['cq']
    qiq[item['seed']] = item['qiq']
    gw[item['seed']] = item['gw']


LINE_WIDTH, MARKER_SIZE = 0.1, 2
print(f'on {sum(1 if cq[i] > max(gw[i], qiq[i]) else 0 for i in range(len(data)))}/{len(data)} instances cq outperforms the other 2')
plt.plot(np.arange(len(data)), cq, 's--', label='QAOA coupled local search', linewidth=LINE_WIDTH, markersize=MARKER_SIZE)
plt.plot(np.arange(len(data)), qiq, 'o--', label='QAOA in QAOA', linewidth=LINE_WIDTH, markersize=MARKER_SIZE)
plt.plot(np.arange(len(data)), gw, 'x--', label='Goemans-Williamson', linewidth=LINE_WIDTH, markersize=MARKER_SIZE)

L, R = -5, 105
ALPHA = 1

AVG_LW = 0.5

avg_ratios = [np.average(cq), np.average(qiq), np.average(gw)]

for i in range(len(avg_ratios)):
    plt.plot([0, 100], [avg_ratios[i], avg_ratios[i]], color=f'C{i}', linewidth=AVG_LW, alpha=ALPHA)
    plt.plot([-5, 0], [avg_ratios[i], avg_ratios[i]], '--', color=f'C{i}', linewidth=AVG_LW, alpha=ALPHA)

ticks = np.concatenate((np.arange(0.93, 1.1, 0.01), avg_ratios))
ticks.sort()

labels = []
for x in ticks:
    if x in avg_ratios:
        labels.append('%.4f' % (x))
    else:
        labels.append('$%.2f$' % (x))

_, label_objs = plt.yticks(ticks, labels)

for i in range(len(ticks)):
    if ticks[i] in avg_ratios:
        label_objs[i].set_fontsize('small')
        label_objs[i].set_color(f'C{avg_ratios.index(ticks[i])}')

plt.xticks(np.arange(0, 110, 10))
plt.xlim(L, R)
plt.ylim(0.925, 1.005)

plt.xlabel('random seed')
plt.ylabel('approximation ratio')
plt.legend()
plt.savefig('fig/cmp.pdf')


on 96/100 instances cq outperforms the other 2


# Demonstrating the increment of $|T_{\rm greedy}|$
The increment of $|T_{\rm greedy}|$ for different settings are listed in `T_greedy-sizes.json`. For each setting, we generate $20$ random graphs, and for each graph, we generate $20$ permutations of $\{0,1,\dots, 2^{n_0}-1\}$, and compute $T_{\rm greedy}$ accordingly.

Results for different graphs with the same setting are averaged and plotted in the following cell.

In [3]:
data = json.load(open('data/T_greedy-sizes.json'))


def plot_with_shade(data, small=False):
    plt.cla()
    ax = formatter.figure(width_ratio=1 if not small else 0.5).gca()
    ax.yaxis.get_major_locator().set_params(integer=True)

    avg, mn, mx = data
    mn = np.concatenate(([1], mn))
    mx = np.concatenate(([1], mx))
    avg = np.concatenate(([1], avg))

    xs = list(range(len(avg)))
    plt.plot(xs, avg)
    plt.fill_between(xs, mn, mx, alpha=.1)

    if not small:
        step = np.ceil(len(xs) / 10).astype(int)
        plt.xlabel('$|U_0|$')
        plt.ylabel('$|T_{\\rm greedy}(U_0)|$')
    else:
        step = np.ceil(len(xs) / 10).astype(int)
    plt.xticks(xs[::step], ['$2^{%d}$' % (x) for x in xs[::step]])


def calc_avg_min_max(d):
    avgs = []
    mn, mx = None, None
    for i in range(len(d['graphs'])):  # iterate all graphs
        x = np.array(d['graphs'][i]['perm_results'])
        # get all results for this graph over different random seeds
        avgs.append(np.average(x, axis=0))
        # compute the average T size for this graph over different random seeds
        if i == 0:
            mn, mx = np.min(x, axis=0), np.max(x, axis=0)
        else:
            mn = np.min(np.array([mn, np.min(x, axis=0)]), axis=0)
            mx = np.max(np.array([mx, np.max(x, axis=0)]), axis=0)
    avgs = np.array(avgs)
    return np.average(avgs, axis=0), mn, mx


def sig_tuple(x):
    return x['graph_size'], x['split_ratio'], x['beta_guarantee']


avg, mn, mx = calc_avg_min_max(data[0])
plot_with_shade((avg, mn, mx), small=False)
plt.plot([0, 27], [1 + 0.1, avg[-1] + 0.1], '--', color='grey')
plt.savefig(f'fig/{sig_tuple(data[0])}-avg.pdf')

for i in range(1, len(data)):
    plot_with_shade(calc_avg_min_max(data[i]), small=True)
    plt.savefig(f'fig/{sig_tuple(data[i])}-avg.pdf')

for seed in range(2):
    x = np.array(data[0]['graphs'][seed]['perm_results'])
    plot_with_shade((np.average(x, axis=0), np.min(x, axis=0), np.max(x, axis=0)), small=True)
    plt.title(f'seed = {seed}')
    plt.savefig(f'fig/{sig_tuple(data[0])}-seed{seed}.pdf')
